In [54]:
from pydantic import BaseModel, Field, TypeAdapter
import json

# If you have the original model class available
class Person(BaseModel):
    name: str = Field(..., description="The person's name")
    age: int = Field(
        ...,
        description="The person's age",
        ge=0,  # age must be non-negative
        le=120,  # age must be less than or equal to 120
    )
    is_student: bool = False

# You can use TypeAdapter for validation
schema = Person.model_json_schema()
print(type(schema))
print(json.dumps(schema, indent=2))

<class 'dict'>
{
  "properties": {
    "name": {
      "description": "The person's name",
      "title": "Name",
      "type": "string"
    },
    "age": {
      "description": "The person's age",
      "maximum": 120,
      "minimum": 0,
      "title": "Age",
      "type": "integer"
    },
    "is_student": {
      "default": false,
      "title": "Is Student",
      "type": "boolean"
    }
  },
  "required": [
    "name",
    "age"
  ],
  "title": "Person",
  "type": "object"
}


In [55]:
data = {"name": "Alice", "age": 0, "is_student": False}
data = json.dumps(data)
print(data)
print('---')
adapter = TypeAdapter(Person)  # PROBLEM: NEED JSON CLASS
validated = adapter.validate_json(data)
validated

{"name": "Alice", "age": 0, "is_student": false}
---


Person(name='Alice', age=0, is_student=False)

In [56]:
from jsonschema import validate, ValidationError

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age":  {"type": "integer", "minimum": 0, "maximum": 120},
    },
    "required": ["name", "age"],
}
data = {"name": "Alice", "age": -1}  # age 130 violates the schema (max 120)

def validate_json_schema(data: dict | str | BaseModel, schema: dict | str | BaseModel) -> tuple[bool, str | None]:
    """Validate data against a JSON schema."""
    if isinstance(schema, BaseModel):
        # TODO I think this should work
        schema = schema.model_json_schema()
    elif isinstance(schema, str):
        schema = json.loads(schema)
    elif not isinstance(schema, dict):
        raise ValueError("Schema must be a dict, JSON string, or Pydantic BaseModel")
    if isinstance(data, str):
        data = json.loads(data)
    elif isinstance(data, BaseModel):
        # TODO I think this should work
        data = data.model_dump()
    elif not isinstance(data, dict):
        raise ValueError("Data must be a dict, JSON string, or Pydantic BaseModel")

    valid = False
    validation_error = None
    try:
        validate(instance=data, schema=schema)
        valid = True
    except ValidationError as e:
        validation_error = str(e)

    return valid, validation_error

valid, validation_error = validate_json_schema(data, schema)
print("Valid: ", valid)
print(f"Validation Error:\n```\n{validation_error}\n```")

Valid:  False
Validation Error:
```
-1 is less than the minimum of 0

Failed validating 'minimum' in schema['properties']['age']:
    {'type': 'integer', 'minimum': 0, 'maximum': 120}

On instance['age']:
    -1
```


In [57]:
valid, validation_error = validate_json_schema(json.dumps(data), json.dumps(schema))
print("Valid: ", valid)
print(f"Validation Error:\n```\n{validation_error}\n```")

Valid:  False
Validation Error:
```
-1 is less than the minimum of 0

Failed validating 'minimum' in schema['properties']['age']:
    {'type': 'integer', 'minimum': 0, 'maximum': 120}

On instance['age']:
    -1
```


In [ ]:
pydanic_schema = Person.model_json_schema()
pydanic_schema = json.dumps(pydanic_schema, indent=2)
print(pydanic_schema)

{
  "properties": {
    "name": {
      "description": "The person's name",
      "title": "Name",
      "type": "string"
    },
    "age": {
      "description": "The person's age",
      "maximum": 120,
      "minimum": 0,
      "title": "Age",
      "type": "integer"
    },
    "is_student": {
      "default": false,
      "title": "Is Student",
      "type": "boolean"
    }
  },
  "required": [
    "name",
    "age"
  ],
  "title": "MyJSONSchema",
  "type": "object"
}


In [53]:
valid, validation_error = validate_json_schema(data, pydanic_schema)
print("Valid: ", valid)
print(f"Validation Error:\n```\n{validation_error}\n```")

Valid:  False
Validation Error:
```
-1 is less than the minimum of 0

Failed validating 'minimum' in schema['properties']['age']:
    {'description': "The person's age",
     'maximum': 120,
     'minimum': 0,
     'title': 'Age',
     'type': 'integer'}

On instance['age']:
    -1
```


In [66]:
import json
from collections.abc import Mapping
from typing import Any

from jsonschema import Draft202012Validator, FormatChecker
from pydantic import BaseModel

JSONLike = Any | str | BaseModel  # keep it simple: one alias for both inputs


def _coerce(value: JSONLike, *, as_schema: bool) -> Any:  # noqa: ANN401
    """
    Coerce input to a Python object suitable for validation.
    - If BaseModel: dump to schema (for schema) or to data (for data).
    - If str: json.loads.
    - Otherwise: pass through.
    Additionally, if as_schema=True, ensure result is a mapping.
    """
    if isinstance(value, BaseModel):
        value = value.__class__.model_json_schema() if as_schema else value.model_dump()
    elif isinstance(value, str):
        value = json.loads(value)

    if as_schema and not isinstance(value, Mapping):
        raise TypeError("Schema must be a JSON object (mapping) after coercion.")
    return value


def validate_json_schema(data: JSONLike, schema: JSONLike) -> tuple[bool, str | None]:
    """
    Validate `data` against JSON `schema`.

    Returns (is_valid, error_report). If invalid, error_report lists all issues.
    """
    schema = _coerce(schema, as_schema=True)
    data = _coerce(data, as_schema=False)

    # Draft202012Validator is the implementation of the JSON Schema 2020-12 draft, which is the
    # latest stable version of the standard.
    validator = Draft202012Validator(schema, format_checker=FormatChecker())
    errors = sorted(validator.iter_errors(data), key=lambda e: list(e.path))

    if not errors:
        return True, None

    validation_errors = []
    for err in errors:
        path_elems = [str(p) for p in err.path]
        path = "/" + "/".join(path_elems) if path_elems else "(root)"
        validation_errors.append(f"{path}: {err.message}")
    return False, validation_errors

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age":  {"type": "integer", "minimum": 0, "maximum": 120},
    },
    "required": ["name", "age"],
    "additionalProperties": False,
}

# data = {"name": "Alice", "age": -1}  # violates minimum 0
data = {'age': -1} # violates required name and minimum age

valid, validation_errors = validate_json_schema(data, schema)
print("Valid:", valid)
print("Validation Errors:")
for err in (validation_errors or []):
    print(err)

Valid: False
Validation Errors:
(root): 'name' is a required property
/age: -1 is less than the minimum of 0
